In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
import sys
sys.path.append('../')

from myutils.metrics import mapk

OUTPUT_NAMEと、item_vecs = make_prone_vec(train_test_log_df, dimensions)を変えるだけ

In [3]:
OUTPUT_NAME = 'prone_train_only'

In [4]:
train_log_df = pd.read_csv('../datasets/atmaCup16_Dataset/train_log.csv')
train_df = pd.read_csv('../datasets/atmaCup16_Dataset/train_label.csv')
test_log_df = pd.read_csv('../datasets/atmaCup16_Dataset/test_log.csv')
yado_df = pd.read_csv('../datasets/atmaCup16_Dataset/yado.csv')

train_session_only_df = train_df[['session_id']]
test_session_only_df = pd.read_csv('../datasets/atmaCup16_Dataset/test_session.csv')

train_test_log_df = pd.concat([train_log_df, test_log_df], axis=0).reset_index(drop=True)

In [5]:
yado_df['yad_no'].min()

1

In [6]:
from csrgraph import csrgraph
import nodevectors as nv
from scipy.sparse import csr_matrix
from nodevectors.prone import ProNE
from nodevectors import prone

def create_adjacency_matrix(edges):
    """
    無向グラフ
    """
    n = np.max(edges) + 1  # ノードの数を取得します（ノードが0から始まると仮定します）

    # edges_flipped = np.flip(edges, axis=1)  # エッジの始点と終点を入れ替えます
    # edges_doubled = np.vstack((edges, edges_flipped))  # エッジリストを2倍にします
    data = np.ones(len(edges))  # エッジの数だけ1を用意します
    adjacency_matrix = csr_matrix((data, (edges[:, 0], edges[:, 1])), shape=(n, n))  # CSR形式のスパース行列を作成します

    return adjacency_matrix


def calc_prone_embs(edge, n_components):
    # 隣接行列
    adj_mat = create_adjacency_matrix(edge)
    # グラフ
    graph = csrgraph(adj_mat)

    # 埋め込みモデルの学習
    model_params = dict(n_components=n_components, step=10, mu=0.2, theta=0.5, exponent=0.75, verbose=True)
    node_emb_model = prone.ProNE(**model_params)
    node_embs = node_emb_model.fit_transform(graph)
    # 正規化する
    node_embs = node_embs / (np.linalg.norm(node_embs, axis=1, keepdims=True) + 1e-8)
    return node_embs


def make_prone_vec(train_test_log_df: pd.DataFrame, n_components:int):
    # 視聴グラフを作成
    # エッジ。anime_noがuser_noの後に続くように調整
    
    # view_edge = df[[f"{name1}_no", f"{name2}_no"]].values
    # num_user = np.max(view_edge[:, 0]) + 1
    # view_edge[:, 1] = view_edge[:, 1] + num_user
    sentence_list = train_test_log_df.groupby('session_id')['yad_no'].apply(list).tolist()

    edges = set()
    for items in sentence_list:
        if len(items) == 1:
            continue
        for i in range(len(items)-1):
            edges.add((items[i]-1, items[i+1]-1))
            edges.add((items[i+1]-1, items[i]-1))
    edges = list(edges)
    edges = np.array(edges)
    print(edges.shape) # (num_edge, 2)

    node_embs = calc_prone_embs(edges, n_components)

    return node_embs

In [7]:
dimensions = 1024

In [8]:
# item_vecs = make_prone_vec(train_test_log_df, dimensions)
item_vecs = make_prone_vec(train_log_df, dimensions)

(90274, 2)


In [9]:
item_vecs.shape

(13806, 1024)

In [10]:
# item_ids = train_test_log_df['yad_no'].unique().tolist()
item_ids = yado_df['yad_no'].to_list()
item_factors = {item: item_vecs[item-1, :] for item in item_ids}
item_factors_df = pd.DataFrame(item_factors).T.reset_index().rename(columns={"index": "yad_no"})

item_cols = [f"item_factor_{i}" for i in range(dimensions)]
item_factors_df.columns = ["yad_no"] + item_cols

In [11]:
user_item_list_dict = {session_id: yad_nos.tolist() for session_id, yad_nos in train_test_log_df.groupby('session_id')['yad_no']}
user_factors = {user_id: np.mean([item_factors[item_id] for item_id in user_item_list], axis=0) for user_id, user_item_list in user_item_list_dict.items()}
user_factors_df = pd.DataFrame(user_factors).T.reset_index().rename(columns={"index": "session_id"})
user_cols = [f"user_factor_{i}" for i in range(dimensions)]
user_factors_df.columns = ["session_id"] + user_cols

user_factors_last_item = {user_id: item_factors[user_item_list[-1]] for user_id, user_item_list in user_item_list_dict.items()}
user_factors_last_item_df = pd.DataFrame(user_factors_last_item).T.reset_index().rename(columns={"index": "session_id"})
user_cols = [f"user_factor_{i}" for i in range(dimensions)]
user_factors_last_item_df.columns = ["session_id"] + user_cols

In [12]:
item_factors_df

,yad_no,item_factor_0,item_factor_1,item_factor_2,item_factor_3,item_factor_4,item_factor_5,item_factor_6,item_factor_7,item_factor_8,...,item_factor_1014,item_factor_1015,item_factor_1016,item_factor_1017,item_factor_1018,item_factor_1019,item_factor_1020,item_factor_1021,item_factor_1022,item_factor_1023
0,1,0.001527,-0.000884,0.003434,0.001110,-0.001386,-0.001761,0.000171,8.107971e-04,-0.000615,...,0.041346,-0.026557,-0.014290,-0.015632,-0.028671,0.073275,-0.016758,-0.020510,-0.014890,-0.014310
1,2,0.000012,-0.000017,-0.000014,0.000012,0.000002,0.000008,-0.000019,9.831374e-06,-0.000022,...,-0.001135,-0.002783,0.001400,-0.000087,-0.000148,-0.000601,-0.000150,-0.000554,-0.004587,0.002644
2,3,-0.369015,-0.328137,0.007335,-0.000883,0.003320,-0.000611,0.000021,-4.451818e-04,0.007094,...,-0.007976,-0.001631,-0.005138,0.004396,-0.010910,0.006864,-0.009035,0.008965,0.011053,0.013903
3,4,-0.066028,-0.058210,0.001383,-0.000112,0.000482,0.000023,-0.000036,2.138162e-05,-0.001054,...,-0.012828,-0.024625,-0.010544,-0.008307,-0.034779,0.006985,-0.007622,-0.002696,0.008167,-0.000797
4,5,0.000023,0.000044,0.000141,0.000009,0.000038,-0.000083,-0.000038,1.305470e-04,-0.000115,...,0.001390,-0.001891,0.002085,0.001683,-0.002940,0.000190,0.002099,-0.001295,0.000307,-0.002133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13801,13802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13802,13803,-0.001039,-0.001177,-0.108233,0.000248,0.000001,0.000009,-0.000023,5.486367e-07,-0.000035,...,0.000702,-0.000217,-0.005520,0.002392,-0.001168,0.002305,0.010616,-0.003567,0.000453,0.000435
13803,13804,-0.000009,-0.000038,0.000041,0.000015,-0.000005,-0.000016,-0.000047,1.551986e-05,0.000022,...,-0.025604,-0.006488,0.004069,0.007314,-0.001100,-0.007858,-0.000081,0.015620,-0.013390,0.024614
13804,13805,-0.000026,0.000016,0.000002,-0.000055,-0.000058,-0.000056,-0.000025,-1.333826e-05,0.000004,...,0.001978,0.001599,-0.008626,-0.005819,-0.003285,-0.000833,-0.001964,0.005860,0.002118,0.002961


In [13]:
user_factors_df

,session_id,user_factor_0,user_factor_1,user_factor_2,user_factor_3,user_factor_4,user_factor_5,user_factor_6,user_factor_7,user_factor_8,...,user_factor_1014,user_factor_1015,user_factor_1016,user_factor_1017,user_factor_1018,user_factor_1019,user_factor_1020,user_factor_1021,user_factor_1022,user_factor_1023
0,000007603d533d30453cc45d0f3d119f,-0.000067,2.825537e-05,0.000414,-0.000161,-0.000464,0.000331,-0.000175,2.405126e-06,-0.000145,...,-0.002148,0.000725,-0.002729,0.009042,-0.005475,0.003007,0.008011,0.013607,0.006609,0.007415
1,00001149e9c73985425197104712478c,-0.003854,-2.348363e-03,-0.000362,0.000150,-0.000127,0.000228,-0.000001,1.597787e-04,-0.002013,...,-0.002488,-0.002655,-0.003502,-0.006012,-0.008775,0.003950,0.004685,-0.001768,0.000091,0.004413
2,0000ca043ed437a1472c9d1d154eb49b,-0.000021,7.326563e-06,0.000055,-0.000042,0.000086,-0.000036,0.000097,1.890689e-01,-0.000063,...,-0.004767,0.014522,-0.027362,-0.029752,0.008795,-0.005677,-0.015096,0.004153,0.007315,0.002768
3,0000d4835cf113316fe447e2f80ba1c8,-0.000043,5.400676e-08,-0.000044,-0.000068,-0.000085,-0.000042,-0.000030,-1.971970e-04,0.000181,...,0.004905,0.032472,-0.006501,-0.028834,-0.029262,0.003493,-0.020653,-0.005010,0.000505,0.000521
4,0000e02747d749a52b7736dfa751e258,-0.075518,-6.384572e-02,0.000605,-0.000175,0.000663,-0.000091,0.000011,-3.264056e-05,0.001227,...,0.002397,-0.009949,-0.000290,0.001689,-0.005390,0.003273,0.006958,-0.002041,0.005950,-0.000478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463393,ffff9a7dcc892875c7a8b821fa436228,-0.356970,4.291028e-01,-0.001139,-0.000090,-0.000217,-0.000043,0.000028,-3.046690e-05,-0.000303,...,-0.008638,0.008613,0.018503,0.009134,0.011032,-0.027052,0.007417,0.015597,-0.008763,0.007914
463394,ffffb1d30300fe17f661941fd085b04b,-0.007881,-5.480031e-03,-0.000012,0.000850,-0.000154,0.000528,0.000044,1.292303e-03,-0.003070,...,0.005431,0.002266,-0.038431,-0.005598,0.014803,-0.009232,-0.029997,-0.010614,-0.012419,0.000114
463395,ffffcd5bc19d62cad5a3815c87818d83,0.000003,-3.156539e-07,-0.000021,0.000012,0.000020,0.000010,-0.000002,5.571872e-07,0.000032,...,-0.019223,0.001329,0.006372,-0.017208,-0.003815,-0.116450,0.031798,-0.003898,0.004260,-0.002353
463396,ffffe984aafd6127ce8e43e3ca40c79d,-0.030154,-2.533613e-02,0.000597,-0.000041,0.000234,-0.000036,0.000065,6.395345e-05,0.000194,...,-0.009782,0.007170,-0.010232,0.003283,0.010321,0.000455,-0.012493,-0.009884,0.005753,0.010248


In [14]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [15]:
from sklearn.neighbors import NearestNeighbors


user_vecs = user_factors_df[[f'user_factor_{i}' for i in range(dimensions)]].values
# user_vecs = user_factors_last_item_df[[f'user_factor_{i}' for i in range(dimensions)]].values
item_vecs = item_factors_df[[f'item_factor_{i}' for i in range(dimensions)]].values

nn = NearestNeighbors(n_neighbors=20, metric='cosine')
nn.fit(item_vecs)
dists, indices = nn.kneighbors(user_vecs)

In [16]:
user_last_item_dict = {user_id: user_item_list[-1] for user_id, user_item_list in user_item_list_dict.items()}

In [17]:
results = []

for i, user_id in enumerate(user_item_list_dict.keys()):
    for j in range(20):
        yad_no = indices[i][j]+1
        if user_last_item_dict[user_id] == yad_no:
            continue

        results.append([user_id, yad_no, 1 - dists[i][j]])
word2vec_sim_df = pd.DataFrame(results, columns=['session_id', 'yad_no', 'word2vec_sim'])

In [18]:
word2vec_sim_df

,session_id,yad_no,word2vec_sim
0,000007603d533d30453cc45d0f3d119f,11882,0.985156
1,000007603d533d30453cc45d0f3d119f,4101,0.980241
2,000007603d533d30453cc45d0f3d119f,3324,0.968728
3,000007603d533d30453cc45d0f3d119f,2808,0.964906
4,000007603d533d30453cc45d0f3d119f,5289,0.786354
...,...,...,...
8816377,fffffa7baf370083ebcdd98f26a7e31a,5800,0.699200
8816378,fffffa7baf370083ebcdd98f26a7e31a,3,0.677568
8816379,fffffa7baf370083ebcdd98f26a7e31a,420,0.603279
8816380,fffffa7baf370083ebcdd98f26a7e31a,11273,0.459202


In [19]:
def make_candidate_word2vec(session_only_df: pd.DataFrame, word2vec_sim_df: pd.DataFrame):
    session_df = session_only_df.merge(word2vec_sim_df, on=['session_id'], how='left')

    session_df = session_df[session_df['yad_no'].notnull()].reset_index(drop=True)
    session_df['yad_no'] = session_df['yad_no'].astype(int)

    session_df['rank'] = session_df.groupby('session_id')['word2vec_sim'].rank(ascending=False, method='min')

    pred_df = session_df.groupby('session_id')['yad_no'].apply(lambda x: x.tolist()).reset_index()
    pred_df = pd.merge(session_only_df, pred_df, on='session_id', how='left')

    pred_df['yad_no'] = pred_df['yad_no'].apply(lambda d: d if isinstance(d, list) else [])

    return session_df, pred_df

In [20]:
train_session_df, train_pred_df = make_candidate_word2vec(train_session_only_df, word2vec_sim_df)
test_session_df, test_pred_df = make_candidate_word2vec(test_session_only_df, word2vec_sim_df)

In [21]:
train_session_df

,session_id,yad_no,word2vec_sim,rank
0,000007603d533d30453cc45d0f3d119f,11882,0.985156,1.0
1,000007603d533d30453cc45d0f3d119f,4101,0.980241,2.0
2,000007603d533d30453cc45d0f3d119f,3324,0.968728,3.0
3,000007603d533d30453cc45d0f3d119f,2808,0.964906,4.0
4,000007603d533d30453cc45d0f3d119f,5289,0.786354,5.0
...,...,...,...,...
5490072,fffffa7baf370083ebcdd98f26a7e31a,5800,0.699200,15.0
5490073,fffffa7baf370083ebcdd98f26a7e31a,3,0.677568,16.0
5490074,fffffa7baf370083ebcdd98f26a7e31a,420,0.603279,17.0
5490075,fffffa7baf370083ebcdd98f26a7e31a,11273,0.459202,18.0


In [22]:
"""
cos_sims = []
for session_id, yad_no in tqdm(zip(train_session_df['session_id'].values, train_session_df['yad_no'].values)):
    session_last_yad_no = user_last_item_dict[session_id]

    sim = cos_sim(item_factors[session_last_yad_no], item_factors[yad_no])
    cos_sims.append(sim)
"""

"\ncos_sims = []\nfor session_id, yad_no in tqdm(zip(train_session_df['session_id'].values, train_session_df['yad_no'].values)):\n    session_last_yad_no = user_last_item_dict[session_id]\n\n    sim = cos_sim(item_factors[session_last_yad_no], item_factors[yad_no])\n    cos_sims.append(sim)\n"

In [23]:
train_pred_df

,session_id,yad_no
0,000007603d533d30453cc45d0f3d119f,"[11882, 4101, 3324, 2808, 5289, 5821, 12941, 1..."
1,0000ca043ed437a1472c9d1d154eb49b,"[8253, 4488, 4909, 4233, 6516, 5416, 1480, 158..."
2,0000d4835cf113316fe447e2f80ba1c8,"[5238, 2290, 6722, 1650, 13637, 4355, 2615, 13..."
3,0000fcda1ae1b2f431e55a7075d1f500,"[13296, 626, 13145, 773, 7872, 11111, 1341, 97..."
4,000104bdffaaad1a1e0a9ebacf585f33,"[96, 7240, 254, 11380, 1284, 902, 5490, 11393,..."
...,...,...
288693,ffff2262d38abdeb247ebd591835dcc9,"[13079, 1092, 13682, 8220, 3725, 600, 825, 131..."
288694,ffff2360540745117193ecadcdc06538,"[3940, 6654, 399, 5299, 513, 8465, 2900, 963, ..."
288695,ffff7fb4617164b2604aaf51c40bf82d,"[7308, 7820, 4398, 3566, 10364, 12240, 9558, 3..."
288696,ffffcd5bc19d62cad5a3815c87818d83,"[570, 9671, 12829, 11091, 12500, 7551, 10619, ..."


## cv
dim=256 0.20479048683487233  
dim=512 0.24105351563621108  
dim=1024 0.284614876566241

In [24]:
mapk(train_df['yad_no'].to_list(), train_pred_df['yad_no'].to_list(), k=10)

0.28634134016246116

In [25]:
train_df

,session_id,yad_no
0,000007603d533d30453cc45d0f3d119f,4101
1,0000ca043ed437a1472c9d1d154eb49b,8253
2,0000d4835cf113316fe447e2f80ba1c8,4863
3,0000fcda1ae1b2f431e55a7075d1f500,1652
4,000104bdffaaad1a1e0a9ebacf585f33,96
...,...,...
288693,ffff2262d38abdeb247ebd591835dcc9,2259
288694,ffff2360540745117193ecadcdc06538,963
288695,ffff7fb4617164b2604aaf51c40bf82d,13719
288696,ffffcd5bc19d62cad5a3815c87818d83,10619


In [26]:
train_session_df.to_pickle(f'../datasets/candidate_df/{OUTPUT_NAME}_train.pkl')
test_session_df.to_pickle(f'../datasets/candidate_df/{OUTPUT_NAME}_test.pkl')

In [27]:
import os
os.makedirs(f'../datasets/{OUTPUT_NAME}_pkl', exist_ok=True)
user_factors_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_user_factors_df.pkl')
user_factors_last_item_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_user_factors_last_item_df.pkl')
item_factors_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_item_factors_df.pkl')